In [88]:
import pandas as pd
import itables
import unicodedata
import sqlite3
import pandas as pd
import missingno as msno
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import sqlite3
import recordlinkage
import itables


- [Integracion](#integracion)
  - [Unificacion entre fuentes](#unificacion-entre-fuentes)
  - [Recuperacion de informacion](#recuperacion-de-informacion)
- [Perfilado](#perfilado)
    - [Conexión a la base de datos ya unificada](#conexión-a-la-base-de-datos-ya-unificada)
    - [Vistazo inicial](#vistazo-inicial)
    - [Generador de reporte automático](#generador-de-reporte-automático)
    - [Completitud](#completitud)
    - [Consistencia](#consistencia)
    - [Precisión](#precisión)
    - [Oportunidad](#oportunidad)
- [Limpieza](#limpieza)
    - [Conexion a unificada](#conexion-a-unificada)
- [Análisis](#análisis)
  - [Relaciones entre variables y patrones de comportamiento](#relaciones-entre-variables-y-patrones-de-comportamiento)
  - [Consultas 2 fuentes de datos](#consultas-2-fuentes-de-datos)
  - [Consultas 1 fuente de datos](#consultas-1-fuente-de-datos)
    - [UNPD](#unpd)
    - [WB](#wb)
  - [Analisis](#analisis)
  - [Conclusiones Generales](#conclusiones-generales)

# Integracion

In [89]:
wb = pd.read_csv("./original_data/wb/WB_ESG.csv")
itables.show(wb)

In [90]:
def estandarizar_texto(texto):
    if pd.isna(texto):
        return ''
    s = str(texto)
    nkfd = unicodedata.normalize('NFKD', s)
    sin_acentos = ''.join(c for c in nkfd if not unicodedata.combining(c))
    sin_espacios = sin_acentos.replace(" ", "_")
    return sin_espacios.upper()
def filtrar_wb(wb: pd.DataFrame) -> pd.DataFrame:
    wb_ = wb[wb['REF_AREA_LABEL'] == 'United States'].copy()
    wb_ = wb_.dropna()
    return wb_
def filtrar_unpd(df  : pd.DataFrame) -> pd.DataFrame:
    df_ = df[df['Location'].isin(['United States of America'])].copy()
    df_ = df_.dropna()
    return df_
def dataframe_to_sql(wb: pd.DataFrame,table:str='INDICADOR_UNIFICADO') -> str:
    cols = wb.columns.tolist()
    rows_sql = []
    for _, row in wb.iterrows():
        vals = []
        for c in cols:
            v = row[c]
            if isinstance(v, str):
                v = v.replace("'", "''")
                vals.append(f"'{v}'")
            else:
                vals.append(str(v))
        rows_sql.append(f"({', '.join(vals)})")

    cols_sql = ', '.join(f'"{c}"' for c in cols)
    values_sql = ',\n'.join(rows_sql)
    return f'INSERT INTO {table} ({cols_sql}) VALUES\n{values_sql};'
def clear_db(path: str):
    conn = sqlite3.connect(path)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS INDICADOR_UNIFICADO")
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS INDICADOR_UNIFICADO (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            fuente TEXT NOT NULL CHECK (
                fuente IN ('WORLD_BANK', 'UNDP')
            ),
            nombre TEXT NOT NULL,
            unidad_medida TEXT NULL,
            tipo_medida TEXT NULL,
            valor REAL NOT NULL,
            anio INTEGER NOT NULL,
            edad_inicio INTEGER NULL,
            edad_fin INTEGER NULL
        );
    ''')
    conn.commit()
    conn.close()
clear_db('main.db')

## Unificacion entre fuentes

In [91]:
rutas_poblacion_unpd = [
    "p1.csv",
    "p2.csv",
    "p3.csv",
    "p4.csv",
    "p5.csv",
]
rutas_mortalidad_unpd = [
    "m1.csv",
    "m2.csv",
    "m3.csv",
    "m4.csv",
]
rutas_familia_unpd = [
    "f1.csv",
    "f2.csv",
    "f3.csv",
    "f4.csv",
    "f5.csv",
]
rutas_planeacion_familiar_unpd = [
    "fp1.csv",
]
# agregar prefijos
rutas_poblacion_unpd = [f"./original_data/unpd/{ruta}" for ruta in rutas_poblacion_unpd]
rutas_mortalidad_unpd = [f"./original_data/unpd/{ruta}" for ruta in rutas_mortalidad_unpd]
rutas_familia_unpd = [f"./original_data/unpd/{ruta}" for ruta in rutas_familia_unpd]
rutas_planeacion_familiar_unpd = [f"./original_data/unpd/{ruta}" for ruta in rutas_planeacion_familiar_unpd]
unpd = []
unpd.extend(
    rutas_familia_unpd + rutas_mortalidad_unpd + rutas_poblacion_unpd + rutas_planeacion_familiar_unpd
)
# Cargar y estandarizar los datos de UNPD
dfs = []
for ruta in unpd:
    df = pd.read_csv(ruta, usecols=['Location','AgeStart','AgeEnd','Time','Value','IndicatorShortName'])
    df = filtrar_unpd(df) 
    df['location'] = df['Location'].apply(
        lambda x: unicodedata.normalize('NFKD', str(x))
                            .encode('ascii', 'ignore')
                            .decode('ascii')
                            .upper()
    )
    dfs.append(df)
unpd_dfs = pd.concat(dfs, ignore_index=True)



In [92]:
unpd_dfs['nombre'] = unpd_dfs['IndicatorShortName'].apply(estandarizar_texto)
unpd_dfs['valor'] = unpd_dfs['Value']
unpd_dfs['anio'] = unpd_dfs['Time']
unpd_dfs['edad_inicio'] = unpd_dfs['AgeStart']
unpd_dfs['edad_fin'] = unpd_dfs['AgeEnd']
unpd_dfs = unpd_dfs[['nombre', 'valor', 'anio', 'edad_inicio', 'edad_fin']]
unpd_dfs['fuente'] = 'UNDP'
unpd_dfs = unpd_dfs.dropna()

In [93]:
itables.show(unpd_dfs)

In [94]:
wb_clean = wb.copy()
wb_clean = filtrar_wb(wb_clean)
wb_clean = wb_clean[['INDICATOR_LABEL', 'UNIT_MEASURE_LABEL', 'UNIT_TYPE', 'OBS_VALUE', 'TIME_PERIOD', 'REF_AREA_LABEL']]
wb_clean['nombre'] = wb_clean['INDICATOR_LABEL'].apply(estandarizar_texto)
wb_clean['unidad_medida'] = wb_clean['UNIT_MEASURE_LABEL'].apply(estandarizar_texto)
wb_clean['tipo_medida'] = wb_clean['UNIT_TYPE'].apply(estandarizar_texto)
wb_clean['valor'] = wb_clean['OBS_VALUE']
wb_clean['anio'] = wb_clean['TIME_PERIOD']
wb_clean = wb_clean[['nombre', 'unidad_medida', 'tipo_medida', 'valor', 'anio']]
wb_clean = wb_clean.dropna()
wb_clean['fuente'] = 'WORLD_BANK'
itables.show(wb_clean)

In [95]:
unpd_canonical = dataframe_to_sql(unpd_dfs)
print(unpd_canonical)

INSERT INTO INDICADOR_UNIFICADO ("nombre", "valor", "anio", "edad_inicio", "edad_fin", "fuente") VALUES
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 14824.0, 1950, 15, 15.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 14824.0, 1950, 15, 15.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 40447.0, 1950, 16, 16.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 40447.0, 1950, 16, 16.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 79014.0, 1950, 17, 17.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 79014.0, 1950, 17, 17.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 129785.0, 1950, 18, 18.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 129785.0, 1950, 18, 18.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE', 179007.0, 1950, 19, 19.0, 'UNDP'),
('LIVE_BIRTHS_BY_AGE

In [96]:
wb_canonical = dataframe_to_sql(wb_clean)
print(wb_canonical)

INSERT INTO INDICADOR_UNIFICADO ("nombre", "unidad_medida", "tipo_medida", "valor", "anio", "fuente") VALUES
('AGRICULTURAL_LAND_(%_OF_LAND_AREA)', 'PERCENTAGE_OF_LAND_AREA', 'RATIO', 48.8602417741752, 1961, 'WORLD_BANK'),
('AGRICULTURAL_LAND_(%_OF_LAND_AREA)', 'PERCENTAGE_OF_LAND_AREA', 'RATIO', 48.4761370286583, 1962, 'WORLD_BANK'),
('AGRICULTURAL_LAND_(%_OF_LAND_AREA)', 'PERCENTAGE_OF_LAND_AREA', 'RATIO', 48.1979395040485, 1963, 'WORLD_BANK'),
('AGRICULTURAL_LAND_(%_OF_LAND_AREA)', 'PERCENTAGE_OF_LAND_AREA', 'RATIO', 47.9325163555688, 1964, 'WORLD_BANK'),
('AGRICULTURAL_LAND_(%_OF_LAND_AREA)', 'PERCENTAGE_OF_LAND_AREA', 'RATIO', 47.589791854097, 1965, 'WORLD_BANK'),
('AGRICULTURAL_LAND_(%_OF_LAND_AREA)', 'PERCENTAGE_OF_LAND_AREA', 'RATIO', 47.5545258413619, 1966, 'WORLD_BANK'),
('AGRICULTURAL_LAND_(%_OF_LAND_AREA)', 'PERCENTAGE_OF_LAND_AREA', 'RATIO', 47.5279944447841, 1967, 'WORLD_BANK'),
('AGRICULTURAL_LAND_(%_OF_LAND_AREA)', 'PERCENTAGE_OF_LAND_AREA', 'RATIO', 47.5812756033436, 1

In [97]:
# cargar los datos en la base de datos
conn = sqlite3.connect('main.db')
cursor = conn.cursor()
cursor.executescript(unpd_canonical)
cursor.executescript(wb_canonical)
conn.commit()
conn.close()


## Recuperacion de informacion

In [98]:
conn = sqlite3.connect('main.db')
df = pd.read_sql_query("SELECT * FROM INDICADOR_UNIFICADO", conn)
print(df.shape)
itables.show(df)
conn.close()

(20122, 9)


# Perfilado
### Conexión a la base de datos ya unificada

In [99]:
conn = sqlite3.connect('main.db')
# Cargar a DataFrame
query = "SELECT * FROM INDICADOR_UNIFICADO"
df_unificado = pd.read_sql_query(query, conn)
conn.close()

# Mostrar las primeras filas para verificar
print("Registros cargados:", len(df_unificado))
df_unificado.head()

Registros cargados: 20122


,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad_inicio,edad_fin
0,1,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
1,2,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
2,3,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
3,4,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
4,5,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,79014.0,1950,17.0,17.0


### Vistazo inicial

In [100]:
df_unificado.head()

,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad_inicio,edad_fin
0,1,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
1,2,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
2,3,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
3,4,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
4,5,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,79014.0,1950,17.0,17.0


In [101]:
df_unificado.tail()

,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad_inicio,edad_fin
20117,20118,WORLD_BANK,VOICE_AND_ACCOUNTABILITY:_ESTIMATE,INDEX,IX,0.995732,2018,NaN,NaN
20118,20119,WORLD_BANK,VOICE_AND_ACCOUNTABILITY:_ESTIMATE,INDEX,IX,0.905322,2019,NaN,NaN
20119,20120,WORLD_BANK,VOICE_AND_ACCOUNTABILITY:_ESTIMATE,INDEX,IX,0.855866,2020,NaN,NaN
20120,20121,WORLD_BANK,VOICE_AND_ACCOUNTABILITY:_ESTIMATE,INDEX,IX,0.878147,2021,NaN,NaN
20121,20122,WORLD_BANK,VOICE_AND_ACCOUNTABILITY:_ESTIMATE,INDEX,IX,0.845122,2022,NaN,NaN


In [102]:
print("Valores nulos por columna (TOTAL):\n", df_unificado.isnull().sum())
print("\nTipos de datos:\n", df_unificado.dtypes)
print("\nFuentes únicas:", df_unificado['fuente'].unique())
print("\nnombres únicos:", df_unificado['nombre'].unique())
print("\nAños únicos:", sorted(df_unificado['anio'].unique()))

Valores nulos por columna (TOTAL):
 id                   0
fuente               0
nombre               0
unidad_medida    17764
tipo_medida      17764
valor                0
anio                 0
edad_inicio       2358
edad_fin          2358
dtype: int64

Tipos de datos:
 id                 int64
fuente            object
nombre            object
unidad_medida     object
tipo_medida       object
valor            float64
anio               int64
edad_inicio      float64
edad_fin         float64
dtype: object

Fuentes únicas: ['UNDP' 'WORLD_BANK']

nombres únicos: ['LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE'
 'CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)'
 'SEX_RATIO_AT_BIRTH_(PER_FEMALE_NEWBORN)' 'TOTAL_FERTILITY_RATE'
 'PROBABILITY_OF_DYING_BY_SINGLE_AGE_AND_BY_SEX'
 'DEATHS_BY_AGE_AND_SEX_-_COMPLETE' 'TOTAL_DEATHS_BY_SEX'
 'LIFE_EXPECTANCY_AT_EXACT_AGES,_EX,_BY_SINGLE_AGE_AND_BY_SEX'
 'CHILD_DEPENDENCY_RATIO' 'OLD-AGE_DEPENDENCY_RATIO'
 'NATURAL_CHANGE_OF_POPULATION

### Generador de reporte automático

In [103]:
profile = ProfileReport(
    df_unificado,
    title="Reporte de Perfilado - Datos UNPD y Banco Mundial UNIFICADOS",
    explorative=True,  
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "kendall": {"calculate": True},
        "phi_k": {"calculate": True},
    },
    missing_diagrams={
        "bar": True,
        "matrix": True,
        "heatmap": True,
    },
)

# Guardar como HTML
profile.to_file("reporte_perfilado_unificado.html")
print("\nReporte generado: 'reporte_perfilado_unificado.html'")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:00<00:00, 151.97it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]


Reporte generado: 'reporte_perfilado_unificado.html'


### Completitud 
Identificar valores faltantes 

In [104]:
# Ya se vieron los faltantes pero obtenemos un promedio
completitud = df_unificado.notna().mean() * 100
print("Porcentaje de completitud por columna:\n", completitud)

msno.bar(df_unificado, figsize=(10, 4))


Porcentaje de completitud por columna:
 id               100.000000
fuente           100.000000
nombre           100.000000
unidad_medida     11.718517
tipo_medida       11.718517
valor            100.000000
anio             100.000000
edad_inicio       88.281483
edad_fin          88.281483
dtype: float64


<Axes: >

### Consistencia
Detectar incoherencias en los datos.

In [105]:
# Valores únicos para las categóricas
#print("Fuentes únicas:", df_unificado['fuente'].unique())
#print("\nUnidades de medida:", df_unificado['unidad_medida'].unique())
#print("\nnombre:", df_unificado['nombre'].unique())
#print("\nTipo de medida:", df_unificado['tipo_medida'].unique())


In [106]:
# Verificar rangos numéricos
print("\nRango de años:", df_unificado['anio'].min(), "-", df_unificado['anio'].max())
print("\nRango de valores:", df_unificado['valor'].describe()[['min', 'max']])

# Inconsistencias en rangos de edad (UNDP)
if 'edad_inicio' in df_unificado.columns:
    edades_invalidas = df_unificado[(df_unificado['edad_inicio'] < 0) | (df_unificado['edad_inicio'] > 100)]
    print("\nRegistros con edades inválidas:", len(edades_invalidas))

# inconsistencias respecto a año
futuros = df_unificado[df_unificado['anio'] > 2024]
print(f"\nRegistros con año posterior a 2024: {len(futuros)}")

"""
if len(futuros) > 0:
    print("\nEjemplo de registros inconsistentes (valores en años futuros):")
    display(futuros[['fuente', 'nombre', 'valor', 'anio']].head())
"""



Rango de años: 1950 - 2030

Rango de valores: min   -1.033640e+03
max    3.556499e+08
Name: valor, dtype: float64

Registros con edades inválidas: 0

Registros con año posterior a 2024: 809


'\nif len(futuros) > 0:\n    print("\nEjemplo de registros inconsistentes (valores en años futuros):")\n    display(futuros[[\'fuente\', \'nombre\', \'valor\', \'anio\']].head())\n'

Duplicados exactos

In [107]:
duplicados_exactos = df_unificado[df_unificado.duplicated(keep=False)]  

print(f"Número de registros duplicados exactos: {len(duplicados_exactos)}")
if not duplicados_exactos.empty:
    print("\nEjemplo de registros duplicados:")
    display(duplicados_exactos.sort_values(by=list(df_unificado.columns)).head())

Número de registros duplicados exactos: 0


Duplicados en coplumnas claves

In [108]:
columnas_clave = ['nombre', 'valor', 'anio']  
duplicados_parciales = df_unificado[df_unificado.duplicated(subset=columnas_clave, keep=False)]

print(df_unificado.shape)
print(f"\nNúmero de registros duplicados (parciales en {columnas_clave}): {len(duplicados_parciales)}")
"""
if not duplicados_parciales.empty:
    print("\nEjemplo de duplicados parciales:")
    display(duplicados_parciales.sort_values(by=columnas_clave).head())
"""

(20122, 9)

Número de registros duplicados (parciales en ['nombre', 'valor', 'anio']): 5776


'\nif not duplicados_parciales.empty:\n    print("\nEjemplo de duplicados parciales:")\n    display(duplicados_parciales.sort_values(by=columnas_clave).head())\n'

In [109]:
conteo_duplicados = df_unificado.groupby(columnas_clave).size().reset_index(name='conteo')
conteo_duplicados = conteo_duplicados[conteo_duplicados['conteo'] > 1]

print("\nRegistros con más de una ocurrencia:")
#print(conteo_duplicados.sort_values('conteo', ascending=False))
conteo_duplicados.sort_values('conteo', ascending=False)


Registros con más de una ocurrencia:


,nombre,valor,anio,conteo
871,"CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)",10.55000,2025,2
10999,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,170875.00000,1961,2
11001,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,171059.00000,2005,2
11002,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,171061.00000,2023,2
11003,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,171187.00000,2019,2
...,...,...,...,...
10045,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,32118.00000,1967,2
10046,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,32193.00000,2028,2
10047,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,32216.00000,2018,2
10048,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,32340.00000,1991,2


### Precisión
Validar que los datos representan información correcta.

In [110]:
# Detección de outliers en 'valor' (método IQR)
Q1 = df_unificado['valor'].quantile(0.25)
Q3 = df_unificado['valor'].quantile(0.75)
IQR = Q3 - Q1
outliers = df_unificado[(df_unificado['valor'] < (Q1 - 1.5*IQR)) | (df_unificado['valor'] > (Q3 + 1.5*IQR))]
print("\nNúmero de outliers en 'valor':", len(outliers))

import seaborn as sns
sns.boxplot(x=df_unificado['valor'])


Número de outliers en 'valor': 3155


<Axes: xlabel='valor'>

In [111]:
outliers_por_anio = df_unificado.groupby('anio')['valor'].apply(
    lambda x: x[(x < (x.quantile(0.25) - 1.5 * (x.quantile(0.75) - x.quantile(0.25)))) | 
                (x > (x.quantile(0.75) + 1.5 * (x.quantile(0.75) - x.quantile(0.25))))]
)

# Conteo de outliers por año
print("Número de outliers por año:")
print(outliers_por_anio.groupby('anio').size())


Número de outliers por año:
anio
1950    2
1951    2
1952    2
1953    2
1954    2
       ..
2026    3
2027    3
2028    3
2029    2
2030    2
Name: valor, Length: 81, dtype: int64


In [112]:
# Filtrar outliers
outliers = df_unificado[
    (df_unificado['valor'] < (Q1 - 1.5 * IQR)) | 
    (df_unificado['valor'] > (Q3 + 1.5 * IQR))
][['nombre', 'valor', 'anio']]

print(f"\n {len(outliers)} outliers en 'valor':")
print(outliers.sort_values(by='valor', ascending=False)) 



 3155 outliers en 'valor':
                                                  nombre        valor  anio
17621                            TOTAL_POPULATION_BY_SEX  355649881.0  2030
17620                            TOTAL_POPULATION_BY_SEX  354050267.5  2029
17619                            TOTAL_POPULATION_BY_SEX  352404085.5  2028
17618                            TOTAL_POPULATION_BY_SEX  350728355.0  2027
17617                            TOTAL_POPULATION_BY_SEX  349035494.0  2026
...                                                  ...          ...   ...
4559   LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...     103079.0  2015
1223   LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...     103059.0  1967
1222   LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...     103059.0  1967
5542   LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...     102931.0  2029
5543   LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...     102931.0  2029

[3155 rows x 3 columns]


Importante notar que los outliers en valor corresponden a que en algunos miden índices y en otros se miden poblaciones totales, tomar en cuenta a la hora de limpieza

In [113]:
absolutos = [
    'LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE',
    'DEATHS_BY_AGE_AND_SEX_-_COMPLETE',
    'TOTAL_DEATHS_BY_SEX',
    'TOTAL_POPULATION_BY_SEX',
    'NET_MIGRATION',
    'NATURAL_CHANGE_OF_POPULATION',
    'TREE_COVER_LOSS_(HECTARES)',
    'CO2_EMISSIONS_(METRIC_TONS_PER_CAPITA)',
    'METHANE_EMISSIONS_(METRIC_TONS_OF_CO2_EQUIVALENT_PER_CAPITA)',
    'NITROUS_OXIDE_EMISSIONS_(METRIC_TONS_OF_CO2_EQUIVALENT_PER_CAPITA)',
    'GHG_NET_EMISSIONS/REMOVALS_BY_LUCF_(MT_OF_CO2_EQUIVALENT)',
    'ANNUAL_FRESHWATER_WITHDRAWALS,_TOTAL_(%_OF_INTERNAL_RESOURCES)',
    'SCIENTIFIC_AND_TECHNICAL_JOURNAL_ARTICLES',
    'PATENT_APPLICATIONS,_RESIDENTS',
    'HOSPITAL_BEDS_(PER_1,000_PEOPLE)',
    'POPULATION_AGES_65_AND_ABOVE_(%_OF_TOTAL_POPULATION)'  
]

df_absolutos = df_unificado[df_unificado['nombre'].isin(absolutos)]
df_absolutos.head()

,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad_inicio,edad_fin
0,1,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
1,2,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
2,3,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
3,4,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
4,5,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,79014.0,1950,17.0,17.0


In [114]:
# Cálculo de IQR
Q1_abs = df_absolutos['valor'].quantile(0.25)
Q3_abs = df_absolutos['valor'].quantile(0.75)
IQR_abs = Q3_abs - Q1_abs

# Detección de outliers
outliers_abs = df_absolutos[
    (df_absolutos['valor'] < (Q1_abs - 1.5 * IQR_abs)) | 
    (df_absolutos['valor'] > (Q3_abs + 1.5 * IQR_abs))
]

# Resultados
print(f"\n🔍 Outliers en Valores Absolutos ({len(outliers_abs)} registros):")
"""
print(outliers_abs[['nombre', 'valor', 'anio', 'fuente']].sort_values('valor', ascending=False))
"""
# Visualización
plt.figure(figsize=(10, 4))
sns.boxplot(data=df_absolutos, x='valor', color='skyblue')
plt.title("Distribución de Valores Absolutos (con Outliers)")
plt.show()


🔍 Outliers en Valores Absolutos (292 registros):


C:\Users\marti\AppData\Local\Temp\ipykernel_18256\4240220253.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [115]:
indices = [
    'CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)',
    'SEX_RATIO_AT_BIRTH_(PER_FEMALE_NEWBORN)',
    'TOTAL_FERTILITY_RATE',
    'LIFE_EXPECTANCY_AT_EXACT_AGES,_EX,_BY_SINGLE_AGE_AND_BY_SEX',
    'CHILD_DEPENDENCY_RATIO',
    'OLD-AGE_DEPENDENCY_RATIO',
    'POPULATION_DENSITY_(PERSONS_PER_SQUARE_KM)',
    'POPULATION_DENSITY_(PEOPLE_PER_SQ._KM_OF_LAND_AREA)',
    'CONTRACEPTIVE_USERS',
    'AGRICULTURAL_LAND_(%_OF_LAND_AREA)',
    'FOREST_AREA_(%_OF_LAND_AREA)',
    'FOOD_PRODUCTION_INDEX_(2014-2016_=_100)',
    'CONTROL_OF_CORRUPTION:_ESTIMATE',
    'ACCESS_TO_CLEAN_FUELS_AND_TECHNOLOGIES_FOR_COOKING_(%_OF_POPULATION)',
    'ENERGY_INTENSITY_LEVEL_OF_PRIMARY_ENERGY_(MJ/$2017_PPP_GDP)',
    'ACCESS_TO_ELECTRICITY_(%_OF_POPULATION)',
    'ELECTRICITY_PRODUCTION_FROM_COAL_SOURCES_(%_OF_TOTAL)',
    'RENEWABLE_ELECTRICITY_OUTPUT_(%_OF_TOTAL_ELECTRICITY_OUTPUT)',
    'RENEWABLE_ENERGY_CONSUMPTION_(%_OF_TOTAL_FINAL_ENERGY_CONSUMPTION)',
    'ENERGY_IMPORTS,_NET_(%_OF_ENERGY_USE)',
    'FOSSIL_FUEL_ENERGY_CONSUMPTION_(%_OF_TOTAL)',
    'ENERGY_USE_(KG_OF_OIL_EQUIVALENT_PER_CAPITA)',
    'PM2.5_AIR_POLLUTION,_MEAN_ANNUAL_EXPOSURE_(MICROGRAMS_PER_CUBIC_METER)',
    'STANDARDISED_PRECIPITATION-EVAPOTRANSPIRATION_INDEX',
    'PROPORTION_OF_BODIES_OF_WATER_WITH_GOOD_AMBIENT_WATER_QUALITY',
    'LEVEL_OF_WATER_STRESS:_FRESHWATER_WITHDRAWAL_AS_A_PROPORTION_OF_AVAILABLE_FRESHWATER_RESOURCES',
    'TERRESTRIAL_AND_MARINE_PROTECTED_AREAS_(%_OF_TOTAL_TERRITORIAL_AREA)',
    'RESEARCH_AND_DEVELOPMENT_EXPENDITURE_(%_OF_GDP)',
    'GOVERNMENT_EFFECTIVENESS:_ESTIMATE',
    'STRENGTH_OF_LEGAL_RIGHTS_INDEX_(0=WEAK_TO_12=STRONG)',
    'INDIVIDUALS_USING_THE_INTERNET_(%_OF_POPULATION)',
    'AGRICULTURE,_FORESTRY,_AND_FISHING,_VALUE_ADDED_(%_OF_GDP)',
    'ADJUSTED_SAVINGS:_NET_FOREST_DEPLETION_(%_OF_GNI)',
    'ADJUSTED_SAVINGS:_NATURAL_RESOURCES_DEPLETION_(%_OF_GNI)',
    'GDP_GROWTH_(ANNUAL_%)',
    'POLITICAL_STABILITY_AND_ABSENCE_OF_VIOLENCE/TERRORISM:_ESTIMATE',
    'RULE_OF_LAW:_ESTIMATE',
    'REGULATORY_QUALITY:_ESTIMATE',
    'ECONOMIC_AND_SOCIAL_RIGHTS_PERFORMANCE_SCORE',
    'SCHOOL_ENROLLMENT,_PRIMARY_AND_SECONDARY_(GROSS),_GENDER_PARITY_INDEX_(GPI)',
    'SCHOOL_ENROLLMENT,_PRIMARY_(%_GROSS)',
    'GOVERNMENT_EXPENDITURE_ON_EDUCATION,_TOTAL_(%_OF_GOVERNMENT_EXPENDITURE)',
    'PROPORTION_OF_SEATS_HELD_BY_WOMEN_IN_NATIONAL_PARLIAMENTS_(%)',
    'CAUSE_OF_DEATH,_BY_COMMUNICABLE_DISEASES_AND_MATERNAL,_PRENATAL_AND_NUTRITION_CONDITIONS_(%_OF_TOTAL)',
    'MORTALITY_RATE,_UNDER-5_(PER_1,000_LIVE_BIRTHS)',
    'PEOPLE_USING_SAFELY_MANAGED_DRINKING_WATER_SERVICES_(%_OF_POPULATION)',
    'PEOPLE_USING_SAFELY_MANAGED_SANITATION_SERVICES_(%_OF_POPULATION)',
    'INCOME_SHARE_HELD_BY_LOWEST_20%',
    'GINI_INDEX',
    'ANNUALIZED_AVERAGE_GROWTH_RATE_IN_PER_CAPITA_REAL_SURVEY_MEAN_CONSUMPTION_OR_INCOME,_TOTAL_POPULATION_(%)',
    'LABOR_FORCE_PARTICIPATION_RATE,_TOTAL_(%_OF_TOTAL_POPULATION_AGES_15-64)_(MODELED_ILO_ESTIMATE)',
    'RATIO_OF_FEMALE_TO_MALE_LABOR_FORCE_PARTICIPATION_RATE_(%)_(MODELED_ILO_ESTIMATE)',
    'UNEMPLOYMENT,_TOTAL_(%_OF_TOTAL_LABOR_FORCE)_(MODELED_ILO_ESTIMATE)',
    'PREVALENCE_OF_UNDERNOURISHMENT_(%_OF_POPULATION)',
    'LIFE_EXPECTANCY_AT_BIRTH,_TOTAL_(YEARS)',
    'FERTILITY_RATE,_TOTAL_(BIRTHS_PER_WOMAN)',
    'UNMET_NEED_FOR_CONTRACEPTION_(%_OF_MARRIED_WOMEN_AGES_15-49)',
    'VOICE_AND_ACCOUNTABILITY:_ESTIMATE',
    'HOSPITAL_BEDS_(PER_1,000_PEOPLE)', 
    'POPULATION_AGES_65_AND_ABOVE_(%_OF_TOTAL_POPULATION)',  
    'ANNUAL_FRESHWATER_WITHDRAWALS,_TOTAL_(%_OF_INTERNAL_RESOURCES)',  
    'CO2_EMISSIONS_(METRIC_TONS_PER_CAPITA)', 
    'METHANE_EMISSIONS_(METRIC_TONS_OF_CO2_EQUIVALENT_PER_CAPITA)',  
    'NITROUS_OXIDE_EMISSIONS_(METRIC_TONS_OF_CO2_EQUIVALENT_PER_CAPITA)'  
]

df_no_absolutos = df_unificado[df_unificado['nombre'].isin(indices)]

In [116]:

Q1_noabs = df_no_absolutos['valor'].quantile(0.25)
Q3_noabs = df_no_absolutos['valor'].quantile(0.75)
IQR_noabs = Q3_noabs - Q1_noabs

# Detección de outliers
outliers_noabs = df_no_absolutos[
    (df_no_absolutos['valor'] < (Q1_noabs - 1.5 * IQR_noabs)) | 
    (df_no_absolutos['valor'] > (Q3_noabs + 1.5 * IQR_noabs))
]


print(f"\n Outliers en Índices/Tasas ({len(outliers_noabs)} registros):")
"""
print(outliers_noabs[['nombre', 'valor', 'anio', 'fuente']].sort_values('valor', ascending=False))
"""

# Visualización
plt.figure(figsize=(10, 4))
sns.boxplot(data=df_no_absolutos, x='valor', color='salmon')
plt.title("Distribución de Índices/Tasas (con Outliers)")
plt.show()


 Outliers en Índices/Tasas (117 registros):


C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2363669700.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Oportunidad 

Verificar que los datos están actualizados.

Acciones recomendadas: eliminar registros que no concuerden en tiempo para ambas fuentes, Eliminar años que estén en el futuro, Eliminar registros que tengan mas outliers en la columna de valores. 

In [117]:
# Año más reciente por fuente
actualizacion = df_unificado.groupby('fuente')['anio'].max()
print("\nAño más reciente por fuente:\n", actualizacion)

# Porcentaje de datos por período
df_unificado['decada'] = (df_unificado['anio'] // 10) * 10
print("\nDistribución por década:\n", df_unificado['decada'].value_counts().sort_index())


Año más reciente por fuente:
 fuente
UNDP          2030
WORLD_BANK    2023
Name: anio, dtype: int64

Distribución por década:
 decada
1950     830
1960     982
1970    1082
1980    1126
1990    4321
2000    4446
2010    4475
2020    2776
2030      84
Name: count, dtype: int64


# Limpieza

### Conexion a unificada

In [118]:
conn = sqlite3.connect(r"./main.db")
query = "SELECT * FROM INDICADOR_UNIFICADO"
df_unificado = pd.read_sql_query(query, conn)
conn.close()

print("Registros cargados:", len(df_unificado))
df_unificado.head()

Registros cargados: 20122


,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad_inicio,edad_fin
0,1,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
1,2,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
2,3,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
3,4,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
4,5,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,79014.0,1950,17.0,17.0


Limpieza de absolutos

In [119]:
absolutos = [
    'LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD)_-_COMPLETE',
    'DEATHS_BY_AGE_AND_SEX_-_COMPLETE',
    'TOTAL_DEATHS_BY_SEX',
    'TOTAL_POPULATION_BY_SEX',
    'NET_MIGRATION',
    'NATURAL_CHANGE_OF_POPULATION',
    'TREE_COVER_LOSS_(HECTARES)',
    'SCIENTIFIC_AND_TECHNICAL_JOURNAL_ARTICLES',
    'PATENT_APPLICATIONS,_RESIDENTS',
    'GHG_NET_EMISSIONS/REMOVALS_BY_LUCF_(MT_OF_CO2_EQUIVALENT)'
]

df_absolutos = df_unificado[df_unificado['nombre'].isin(absolutos)].copy()
df_absolutos.head()

,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad_inicio,edad_fin
0,1,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
1,2,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,14824.0,1950,15.0,15.0
2,3,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
3,4,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,40447.0,1950,16.0,16.0
4,5,UNDP,LIVE_BIRTHS_BY_AGE_OF_MOTHER_(AND_SEX_OF_CHILD...,None,None,79014.0,1950,17.0,17.0


In [120]:
df_absolutos['edad_inicio'] = pd.to_numeric(df_absolutos['edad_inicio'], errors='coerce')
df_absolutos['edad_fin'] = pd.to_numeric(df_absolutos['edad_fin'], errors='coerce')

# Calcular la diferencia de edad
diferencia = df_absolutos['edad_fin'] - df_absolutos['edad_inicio']

# Condiciones
cond_misma_edad = (diferencia >= 0) & (diferencia <= 1)
cond_no_importa_edad = (diferencia > 1) | (diferencia < 0) | (df_absolutos['edad_inicio'].isna()) | (df_absolutos['edad_fin'].isna())

# Dividir en los dos DataFrames
df_absolutos_misma_edad = df_absolutos[cond_misma_edad].copy()
df_absolutos_no_importa_edad = df_absolutos[cond_no_importa_edad].copy()

In [121]:
df_absolutos_misma_edad = df_absolutos_misma_edad.drop(columns=['edad_fin'])
df_absolutos_misma_edad = df_absolutos_misma_edad.rename(columns={'edad_inicio': 'edad'})
df_absolutos_no_importa_edad = df_absolutos_no_importa_edad.drop(columns=['edad_inicio', 'edad_fin'])

- Vamos a eliminar los valores de los años que esten despues del actual (es decir, 2025) para que no afecte en nuestro analisis. 

In [122]:
df_absolutos_no_importa_edad = df_absolutos_no_importa_edad[df_absolutos_no_importa_edad['anio'] <= 2025]
df_absolutos_misma_edad = df_absolutos_misma_edad[df_absolutos_misma_edad['anio'] <= 2025]

- Agregar datos faltantes 

In [123]:
df_absolutos_misma_edad.loc[:, 'unidad_medida'] = df_absolutos_misma_edad['unidad_medida'].fillna("Desconocido")
df_absolutos_misma_edad.loc[:, 'tipo_medida'] = df_absolutos_misma_edad['tipo_medida'].fillna("Desconocido")

df_absolutos_no_importa_edad.loc[:, 'unidad_medida'] = df_absolutos_no_importa_edad['unidad_medida'].fillna("Desconocido")
df_absolutos_no_importa_edad.loc[:, 'tipo_medida'] = df_absolutos_no_importa_edad['tipo_medida'].fillna("Desconocido")


- Patrones en texto(nombre, unidad)

In [124]:
# Misma edad
df_absolutos_misma_edad.loc[:, 'nombre'] = df_absolutos_misma_edad['nombre'].str.strip().str.title()
df_absolutos_misma_edad.loc[:, 'unidad_medida'] = df_absolutos_misma_edad['unidad_medida'].str.lower().str.strip()
df_absolutos_misma_edad.loc[:, 'tipo_medida'] = df_absolutos_misma_edad['tipo_medida'].str.lower().str.strip()

# No importa edad
df_absolutos_no_importa_edad.loc[:, 'nombre'] = df_absolutos_no_importa_edad['nombre'].str.strip().str.title()
df_absolutos_no_importa_edad.loc[:, 'unidad_medida'] = df_absolutos_no_importa_edad['unidad_medida'].str.lower().str.strip()
df_absolutos_no_importa_edad.loc[:, 'tipo_medida'] = df_absolutos_no_importa_edad['tipo_medida'].str.lower().str.strip()

In [125]:
df_absolutos_no_importa_edad.head()

,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio
13014,13015,UNDP,Total_Deaths_By_Sex,desconocido,desconocido,2178344.0,1990
13015,13016,UNDP,Total_Deaths_By_Sex,desconocido,desconocido,2199356.0,1991
13016,13017,UNDP,Total_Deaths_By_Sex,desconocido,desconocido,2203669.0,1992
13017,13018,UNDP,Total_Deaths_By_Sex,desconocido,desconocido,2295668.0,1993
13018,13019,UNDP,Total_Deaths_By_Sex,desconocido,desconocido,2305017.0,1994


In [126]:
df_absolutos_misma_edad.head()

,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad
0,1,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,14824.0,1950,15.0
1,2,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,14824.0,1950,15.0
2,3,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,40447.0,1950,16.0
3,4,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,40447.0,1950,16.0
4,5,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,79014.0,1950,17.0


In [127]:
def detectar_duplicados_por_lotes(df, batch_size=1000):
    duplicados_ids = set()
    for start in range(0, len(df), batch_size):
        end = start + batch_size
        df_lote = df.iloc[start:end]

        indexer = recordlinkage.Index()
        indexer.full()
        candidate_links = indexer.index(df_lote)

        compare = recordlinkage.Compare()
        compare.string('nombre', 'nombre', method='jarowinkler', threshold=0.95, label='nombre_similar')
        compare.exact('anio', 'anio', label='anio_igual')
        compare.numeric('valor', 'valor', offset=0.01, scale=1.0, label='valor_cercano')
        compare.numeric('edad', 'edad', offset=1, scale=1.0, label='edad_cercano')

        features = compare.compute(candidate_links, df_lote)
        features['score_total'] = features.sum(axis=1)
        duplicados = features[features["score_total"] >= 4].index
        duplicados_ids.update(i for i, _ in duplicados)

    return df.drop(index=duplicados_ids, errors='ignore')

# Ejecutar
df_absolutos_limpio_edad = detectar_duplicados_por_lotes(df_absolutos_misma_edad)

- Discretizacion 

In [128]:
bins = [0, 5, 12, 18, 30, 60, 100]
labels = ['Bebé', 'Niñez', 'Adolescencia', 'Joven', 'Adulto', 'Mayor']
df_absolutos_limpio_edad['edad_grupo'] = pd.cut(df_absolutos_limpio_edad['edad'], bins=bins, labels=labels)

- Normalizacion(Min-Max)

In [129]:
df_absolutos_limpio_edad['valor_norm'] = df_absolutos_limpio_edad.groupby('nombre')['valor'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
itables.show(df_absolutos_limpio_edad)

In [130]:
df_absolutos_no_importa_edad['valor_norm'] = df_absolutos_no_importa_edad.groupby('nombre')['valor'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
itables.show(df_absolutos_no_importa_edad)

In [131]:
# Generar el archivo CSV
df_absolutos_limpio_edad.to_csv('./cleaned_data/absolutos_misma_edad.csv', index=False)
df_absolutos_no_importa_edad.to_csv('./cleaned_data/absolutos_no_importa_edad.csv', index=False)

Limpieza de indices 

In [132]:
indices = [
    'CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)',
    'SEX_RATIO_AT_BIRTH_(PER_FEMALE_NEWBORN)',
    'TOTAL_FERTILITY_RATE',
    'LIFE_EXPECTANCY_AT_EXACT_AGES,_EX,_BY_SINGLE_AGE_AND_BY_SEX',
    'CHILD_DEPENDENCY_RATIO',
    'OLD-AGE_DEPENDENCY_RATIO',
    'POPULATION_DENSITY_(PERSONS_PER_SQUARE_KM)',
    'POPULATION_DENSITY_(PEOPLE_PER_SQ._KM_OF_LAND_AREA)',
    'CONTRACEPTIVE_USERS',
    'AGRICULTURAL_LAND_(%_OF_LAND_AREA)',
    'FOREST_AREA_(%_OF_LAND_AREA)',
    'FOOD_PRODUCTION_INDEX_(2014-2016_=_100)',
    'CONTROL_OF_CORRUPTION:_ESTIMATE',
    'ACCESS_TO_CLEAN_FUELS_AND_TECHNOLOGIES_FOR_COOKING_(%_OF_POPULATION)',
    'ENERGY_INTENSITY_LEVEL_OF_PRIMARY_ENERGY_(MJ/$2017_PPP_GDP)',
    'ACCESS_TO_ELECTRICITY_(%_OF_POPULATION)',
    'ELECTRICITY_PRODUCTION_FROM_COAL_SOURCES_(%_OF_TOTAL)',
    'RENEWABLE_ELECTRICITY_OUTPUT_(%_OF_TOTAL_ELECTRICITY_OUTPUT)',
    'RENEWABLE_ENERGY_CONSUMPTION_(%_OF_TOTAL_FINAL_ENERGY_CONSUMPTION)',
    'ENERGY_IMPORTS,_NET_(%_OF_ENERGY_USE)',
    'FOSSIL_FUEL_ENERGY_CONSUMPTION_(%_OF_TOTAL)',
    'ENERGY_USE_(KG_OF_OIL_EQUIVALENT_PER_CAPITA)',
    'PM2.5_AIR_POLLUTION,_MEAN_ANNUAL_EXPOSURE_(MICROGRAMS_PER_CUBIC_METER)',
    'STANDARDISED_PRECIPITATION-EVAPOTRANSPIRATION_INDEX',
    'PROPORTION_OF_BODIES_OF_WATER_WITH_GOOD_AMBIENT_WATER_QUALITY',
    'LEVEL_OF_WATER_STRESS:_FRESHWATER_WITHDRAWAL_AS_A_PROPORTION_OF_AVAILABLE_FRESHWATER_RESOURCES',
    'TERRESTRIAL_AND_MARINE_PROTECTED_AREAS_(%_OF_TOTAL_TERRITORIAL_AREA)',
    'RESEARCH_AND_DEVELOPMENT_EXPENDITURE_(%_OF_GDP)',
    'GOVERNMENT_EFFECTIVENESS:_ESTIMATE',
    'STRENGTH_OF_LEGAL_RIGHTS_INDEX_(0=WEAK_TO_12=STRONG)',
    'INDIVIDUALS_USING_THE_INTERNET_(%_OF_POPULATION)',
    'AGRICULTURE,_FORESTRY,_AND_FISHING,_VALUE_ADDED_(%_OF_GDP)',
    'ADJUSTED_SAVINGS:_NET_FOREST_DEPLETION_(%_OF_GNI)',
    'ADJUSTED_SAVINGS:_NATURAL_RESOURCES_DEPLETION_(%_OF_GNI)',
    'GDP_GROWTH_(ANNUAL_%)',
    'POLITICAL_STABILITY_AND_ABSENCE_OF_VIOLENCE/TERRORISM:_ESTIMATE',
    'RULE_OF_LAW:_ESTIMATE',
    'REGULATORY_QUALITY:_ESTIMATE',
    'ECONOMIC_AND_SOCIAL_RIGHTS_PERFORMANCE_SCORE',
    'SCHOOL_ENROLLMENT,_PRIMARY_AND_SECONDARY_(GROSS),_GENDER_PARITY_INDEX_(GPI)',
    'SCHOOL_ENROLLMENT,_PRIMARY_(%_GROSS)',
    'GOVERNMENT_EXPENDITURE_ON_EDUCATION,_TOTAL_(%_OF_GOVERNMENT_EXPENDITURE)',
    'PROPORTION_OF_SEATS_HELD_BY_WOMEN_IN_NATIONAL_PARLIAMENTS_(%)',
    'CAUSE_OF_DEATH,_BY_COMMUNICABLE_DISEASES_AND_MATERNAL,_PRENATAL_AND_NUTRITION_CONDITIONS_(%_OF_TOTAL)',
    'MORTALITY_RATE,_UNDER-5_(PER_1,000_LIVE_BIRTHS)',
    'PEOPLE_USING_SAFELY_MANAGED_DRINKING_WATER_SERVICES_(%_OF_POPULATION)',
    'PEOPLE_USING_SAFELY_MANAGED_SANITATION_SERVICES_(%_OF_POPULATION)',
    'INCOME_SHARE_HELD_BY_LOWEST_20%',
    'GINI_INDEX',
    'ANNUALIZED_AVERAGE_GROWTH_RATE_IN_PER_CAPITA_REAL_SURVEY_MEAN_CONSUMPTION_OR_INCOME,_TOTAL_POPULATION_(%)',
    'LABOR_FORCE_PARTICIPATION_RATE,_TOTAL_(%_OF_TOTAL_POPULATION_AGES_15-64)_(MODELED_ILO_ESTIMATE)',
    'RATIO_OF_FEMALE_TO_MALE_LABOR_FORCE_PARTICIPATION_RATE_(%)_(MODELED_ILO_ESTIMATE)',
    'UNEMPLOYMENT,_TOTAL_(%_OF_TOTAL_LABOR_FORCE)_(MODELED_ILO_ESTIMATE)',
    'PREVALENCE_OF_UNDERNOURISHMENT_(%_OF_POPULATION)',
    'LIFE_EXPECTANCY_AT_BIRTH,_TOTAL_(YEARS)',
    'FERTILITY_RATE,_TOTAL_(BIRTHS_PER_WOMAN)',
    'UNMET_NEED_FOR_CONTRACEPTION_(%_OF_MARRIED_WOMEN_AGES_15-49)',
    'VOICE_AND_ACCOUNTABILITY:_ESTIMATE',
    'HOSPITAL_BEDS_(PER_1,000_PEOPLE)', 
    'POPULATION_AGES_65_AND_ABOVE_(%_OF_TOTAL_POPULATION)',  
    'ANNUAL_FRESHWATER_WITHDRAWALS,_TOTAL_(%_OF_INTERNAL_RESOURCES)',  
    'CO2_EMISSIONS_(METRIC_TONS_PER_CAPITA)', 
    'METHANE_EMISSIONS_(METRIC_TONS_OF_CO2_EQUIVALENT_PER_CAPITA)',  
    'NITROUS_OXIDE_EMISSIONS_(METRIC_TONS_OF_CO2_EQUIVALENT_PER_CAPITA)'  
]

df_indices = df_unificado[df_unificado['nombre'].isin(indices)]
df_indices.head()

,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad_inicio,edad_fin
5670,5671,UNDP,"CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)",None,None,16.466,1990,0.0,-1.0
5671,5672,UNDP,"CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)",None,None,16.043,1991,0.0,-1.0
5672,5673,UNDP,"CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)",None,None,15.602,1992,0.0,-1.0
5673,5674,UNDP,"CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)",None,None,15.116,1993,0.0,-1.0
5674,5675,UNDP,"CRUDE_BIRTH_RATE_(BIRTHS_PER_1,000_POPULATION)",None,None,14.713,1994,0.0,-1.0


In [133]:
df_indices.loc[:, 'edad_inicio'] = pd.to_numeric(df_indices['edad_inicio'], errors='coerce')
df_indices.loc[:, 'edad_fin'] = pd.to_numeric(df_indices['edad_fin'], errors='coerce')

# Calcular la diferencia de edad
diferencia = df_indices['edad_fin'] - df_indices['edad_inicio']

# Condiciones
cond_misma_edad = (diferencia >= 0) & (diferencia <= 1)
cond_no_importa_edad = (diferencia > 1) | (diferencia < 0) | (df_indices['edad_inicio'].isna()) | (df_indices['edad_fin'].isna())

# Dividir en los dos DataFrames
df_indices_misma_edad = df_indices[cond_misma_edad].copy()
df_indices_no_importa_edad = df_indices[cond_no_importa_edad].copy()


In [134]:
df_indices_misma_edad = df_indices_misma_edad.drop(columns=['edad_fin'])
df_indices_misma_edad = df_indices_misma_edad.rename(columns={'edad_inicio': 'edad'})
df_indices_no_importa_edad = df_indices_no_importa_edad.drop(columns=['edad_inicio', 'edad_fin'])
df_indices_no_importa_edad = df_indices_no_importa_edad[df_indices_no_importa_edad['anio'] <= 2025]
df_indices_misma_edad = df_indices_misma_edad[df_indices_misma_edad['anio'] <= 2025]

In [135]:
# Rellenar nulos
df_indices_misma_edad.loc[:, 'unidad_medida'] = df_indices_misma_edad['unidad_medida'].fillna("Desconocido")
df_indices_misma_edad.loc[:, 'tipo_medida'] = df_indices_misma_edad['tipo_medida'].fillna("Desconocido")
df_indices_no_importa_edad.loc[:, 'unidad_medida'] = df_indices_no_importa_edad['unidad_medida'].fillna("Desconocido")
df_indices_no_importa_edad.loc[:, 'tipo_medida'] = df_indices_no_importa_edad['tipo_medida'].fillna("Desconocido")

# Limpiar texto
df_indices_misma_edad.loc[:, 'nombre'] = df_indices_misma_edad['nombre'].str.strip().str.title()
df_indices_misma_edad.loc[:, 'unidad_medida'] = df_indices_misma_edad['unidad_medida'].str.lower().str.strip()
df_indices_misma_edad.loc[:, 'tipo_medida'] = df_indices_misma_edad['tipo_medida'].str.lower().str.strip()

df_indices_no_importa_edad.loc[:, 'nombre'] = df_indices_no_importa_edad['nombre'].str.strip().str.title()
df_indices_no_importa_edad.loc[:, 'unidad_medida'] = df_indices_no_importa_edad['unidad_medida'].str.lower().str.strip()
df_indices_no_importa_edad.loc[:, 'tipo_medida'] = df_indices_no_importa_edad['tipo_medida'].str.lower().str.strip()

In [136]:
df_indices_misma_edad['valor_norm'] = df_indices_misma_edad.groupby('nombre')['valor'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
itables.show(df_indices_misma_edad)

In [137]:
df_indices_no_importa_edad['valor_norm'] = df_indices_no_importa_edad.groupby('nombre')['valor'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
itables.show(df_indices_no_importa_edad)

In [138]:
# Generar el archivo CSV
df_indices_misma_edad.to_csv('./cleaned_data/indices_misma_edad.csv', index=False)
df_indices_no_importa_edad.to_csv('./cleaned_data/indices_no_importa_edad.csv', index=False)

# Análisis

In [139]:
# Cargar los 4 archivos
df1 = pd.read_csv('./cleaned_data/absolutos_misma_edad.csv')
df2 = pd.read_csv('./cleaned_data/absolutos_no_importa_edad.csv')
df3 = pd.read_csv('./cleaned_data/indices_misma_edad.csv')
df4 = pd.read_csv('./cleaned_data/indices_no_importa_edad.csv')

# Unir todos los DataFrames en uno solo (verticalmente)
df_unificado = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Verificar el resultado
print(f"Total de registros: {len(df_unificado)}")

Total de registros: 13212


In [140]:
df_unificado

,id,fuente,nombre,unidad_medida,tipo_medida,valor,anio,edad,edad_grupo,valor_norm
0,1,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,14824.000000,1950,15.0,Adolescencia,0.045707
1,3,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,40447.000000,1950,16.0,Adolescencia,0.124802
2,5,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,79014.000000,1950,17.0,Adolescencia,0.243853
3,7,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,129785.000000,1950,18.0,Adolescencia,0.400575
4,9,UNDP,Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child...,desconocido,desconocido,179007.000000,1950,19.0,Joven,0.552517
...,...,...,...,...,...,...,...,...,...,...
13207,20118,WORLD_BANK,Voice_And_Accountability:_Estimate,index,ix,0.995732,2018,NaN,NaN,0.299103
13208,20119,WORLD_BANK,Voice_And_Accountability:_Estimate,index,ix,0.905322,2019,NaN,NaN,0.119554
13209,20120,WORLD_BANK,Voice_And_Accountability:_Estimate,index,ix,0.855866,2020,NaN,NaN,0.021337
13210,20121,WORLD_BANK,Voice_And_Accountability:_Estimate,index,ix,0.878147,2021,NaN,NaN,0.065585


In [141]:
pivot_misma_edad = df_unificado.pivot_table(
    index=['anio', 'edad'],  # Filas: tiempo y demografía
    columns='nombre',                           # Columnas: cada indicador
    values='valor',                             # Valores: los números a analizar
    aggfunc='first'                             # Usar el primer valor si hay duplicados
).reset_index()

In [142]:
pivot_misma_edad

nombre,anio,edad,Deaths_By_Age_And_Sex_-_Complete,"Life_Expectancy_At_Exact_Ages,_Ex,_By_Single_Age_And_By_Sex",Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child)_-_Complete
0,1950,15.0,NaN,NaN,14824.0
1,1950,16.0,NaN,NaN,40447.0
2,1950,17.0,NaN,NaN,79014.0
3,1950,18.0,NaN,NaN,129785.0
4,1950,19.0,NaN,NaN,179007.0
...,...,...,...,...,...
4995,2025,95.0,46389.0,3.1699,NaN
4996,2025,96.0,37895.0,2.9409,NaN
4997,2025,97.0,30130.0,2.7320,NaN
4998,2025,98.0,22857.0,2.5424,NaN


In [143]:
pivot_no_importa_edad = df_unificado.pivot_table(
    index='anio',                                # Filas: tiempo
    columns='nombre',                           # Columnas: cada indicador
    values='valor',                             # Valores: los números a analizar
    aggfunc='first'                             # Usar el primer valor si hay duplicados
).reset_index()

In [144]:
pivot_no_importa_edad

nombre,anio,Access_To_Clean_Fuels_And_Technologies_For_Cooking_(%_Of_Population),Access_To_Electricity_(%_Of_Population),Adjusted_Savings:_Natural_Resources_Depletion_(%_Of_Gni),Adjusted_Savings:_Net_Forest_Depletion_(%_Of_Gni),Agricultural_Land_(%_Of_Land_Area),"Agriculture,_Forestry,_And_Fishing,_Value_Added_(%_Of_Gdp)","Annual_Freshwater_Withdrawals,_Total_(%_Of_Internal_Resources)","Annualized_Average_Growth_Rate_In_Per_Capita_Real_Survey_Mean_Consumption_Or_Income,_Total_Population_(%)","Cause_Of_Death,_By_Communicable_Diseases_And_Maternal,_Prenatal_And_Nutrition_Conditions_(%_Of_Total)",...,Standardised_Precipitation-Evapotranspiration_Index,Strength_Of_Legal_Rights_Index_(0=Weak_To_12=Strong),Terrestrial_And_Marine_Protected_Areas_(%_Of_Total_Territorial_Area),Total_Deaths_By_Sex,Total_Fertility_Rate,Total_Population_By_Sex,Tree_Cover_Loss_(Hectares),"Unemployment,_Total_(%_Of_Total_Labor_Force)_(Modeled_Ilo_Estimate)",Unmet_Need_For_Contraception_(%_Of_Married_Women_Ages_15-49),Voice_And_Accountability:_Estimate
0,1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,154202680.5,NaN,NaN,NaN,NaN
1,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,156481323.0,NaN,NaN,NaN,NaN
2,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,158794222.0,NaN,NaN,NaN,NaN
3,1953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,161168315.5,NaN,NaN,NaN,NaN
4,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,163636308.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2021,100.0,100.0,0.815193,0.0,44.363367,0.959567,NaN,2.98,NaN,...,-0.233876,NaN,15.883274,3492881.0,1.633919,340161441.0,2044299.0,5.350,NaN,0.878147
72,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,15.883337,3187302.0,1.665000,341534045.5,NaN,3.650,NaN,0.845122
73,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2975658.0,1.623609,343477335.0,NaN,3.648,NaN,NaN
74,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3046569.0,1.622090,345426571.0,NaN,NaN,NaN,NaN


## Relaciones entre variables y patrones de comportamiento

Heatmap de correlación entre variables clave

In [145]:
# 1. Pivotar los datos
df_pivoted = df_unificado.pivot_table(
    index='anio',
    columns='nombre',
    values='valor',
    aggfunc='mean'
)

# 2. Seleccionar solo columnas numéricas y calcular correlación
numeric_cols = df_pivoted.select_dtypes(include=['float64', 'int64'])
corr_matrix = numeric_cols.corr()

# 3. Filtrar correlaciones débiles (|r| < 0.5)
mask = (corr_matrix.abs() <= 0.5) & (corr_matrix != 1)  # Conservar diagonal principal

# 4. Configurar el heatmap
plt.figure(figsize=(20, 18))
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot_kws={'size': 8},
    fmt=".2f",           # 2 decimales
    cmap='coolwarm',
    center=0,
    vmin=-1,
    vmax=1,
    linewidths=0.5,
    cbar_kws={'shrink': 0.8},
    square=True
)

# 5. Ajustes visuales
plt.title('Correlaciones Fuertes (|r| > 0.5) entre Indicadores', fontsize=16, pad=20)
plt.xticks(ha='right', fontsize=9)
plt.yticks(fontsize=9)
plt.tight_layout()

plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2075255146.py:38: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [146]:
# Seleccionar variables numéricas
variables = [
    'Co2_Emissions_(Metric_Tons_Per_Capita)',
    'Gdp_Growth_(Annual_%)',
    'Life_Expectancy_At_Birth,_Total_(Years)',
    'Forest_Area_(%_Of_Land_Area)'
]
corr_matrix = pivot_no_importa_edad[variables].corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlación entre indicadores clave")
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\13290281.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Consultas 2 fuentes de datos

1. Esperanza de vida vs. CO2 per cápita

In [147]:
# Consulta

consulta_1 = df_unificado[
    (df_unificado['nombre'].isin([
        'Life_Expectancy_At_Birth,_Total_(Years)',  # UNDP
        'Co2_Emissions_(Metric_Tons_Per_Capita)'    # WB
    ]))
].pivot_table(index='anio', columns='nombre', values='valor').reset_index()

# Gráfico
sns.scatterplot(data=consulta_1, x='Co2_Emissions_(Metric_Tons_Per_Capita)', y='Life_Expectancy_At_Birth,_Total_(Years)')
plt.title("EE.UU.: Esperanza de vida vs Emisiones de CO2")
plt.xlabel("Emisiones de CO2 (toneladas métricas per cápita)")
plt.ylabel("Esperanza de vida al nacer (años)")
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\15776488.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


2. Población vs. Acceso a electricidad

In [148]:
consulta_2 = df_unificado[
    df_unificado['nombre'].isin([
        'Total_Population_By_Sex', 
        'Access_To_Electricity_(%_Of_Population)'
    ])
].pivot(index='anio', columns='nombre', values='valor').reset_index()

# Gráfico de líneas doble eje
fig, ax1 = plt.subplots()
ax1.plot(consulta_2['anio'], consulta_2['Total_Population_By_Sex'], 'b-', label='Población')
ax1.set_ylabel('Población')
ax2 = ax1.twinx()
ax2.plot(consulta_2['anio'], consulta_2['Access_To_Electricity_(%_Of_Population)'], 'r-', label='Acceso a electricidad (%)')
ax2.set_ylabel('Acceso a electricidad (%)')
plt.title("EE.UU.: Población vs Acceso a electricidad")
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\830057936.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


3. Fertilidad vs. Educación

In [149]:
consulta_3 = df_unificado[
    df_unificado['nombre'].isin([
        'Total_Fertility_Rate', 
        'Government_Expenditure_On_Education,_Total_(%_Of_Government_Expenditure)'
    ]) & (df_unificado['anio'] % 10 == 0)
].pivot(index='anio', columns='nombre', values='valor').reset_index()

# Gráfico de barras apiladas
consulta_3.plot(x='anio', y=['Total_Fertility_Rate', 'Government_Expenditure_On_Education,_Total_(%_Of_Government_Expenditure)'], kind='bar', stacked=False)
plt.title("EE.UU.: Fertilidad vs Gasto en educación")
plt.xlabel("Año")
plt.ylabel("Tasa de fertilidad / Gasto en educación (%)")
plt.xticks(rotation=45)
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\1373631661.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


4. Mortalidad infantil vs. Acceso a agua potable

In [150]:
consulta_4 = df_unificado[
    df_unificado['nombre'].isin([
        'Mortality_Rate,_Under-5_(Per_1,000_Live_Births)',
        'People_Using_Safely_Managed_Drinking_Water_Services_(%_Of_Population)'
    ])
].pivot(index='anio', columns='nombre', values='valor').reset_index()

sns.regplot(data=consulta_4, x='People_Using_Safely_Managed_Drinking_Water_Services_(%_Of_Population)', y='Mortality_Rate,_Under-5_(Per_1,000_Live_Births)')
plt.title("EE.UU.: Mortalidad infantil vs Acceso a agua potable segura")
plt.xlabel("Acceso a agua potable segura (% población)")
plt.ylabel("Mortalidad infantil (por 1,000 nacidos vivos)")
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2607573628.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


5. Emisiones de CO2 vs. Poblacion Total

In [151]:
sns.scatterplot(data=pivot_no_importa_edad, 
                x='Total_Population_By_Sex', 
                y='Co2_Emissions_(Metric_Tons_Per_Capita)')
plt.title("EE.UU.: Emisiones de CO2 per cápita vs Población Total")
plt.xlabel("Población Total por Sexo")
plt.ylabel("Emisiones de CO2 (Toneladas Métricas por Cápita)")
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2817664055.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


6. Densidad poblacional vs area forestal

In [152]:
sns.scatterplot(data=pivot_no_importa_edad,
           x='Population_Density_(Persons_Per_Square_Km)',
           y='Forest_Area_(%_Of_Land_Area)')
plt.title("EE.UU.: Densidad Poblacional vs Área Forestal")
plt.xlabel("Densidad poblacional (personas/km²)")
plt.ylabel("Área forestal (% del área terrestre)")
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2791016065.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


7. Salud vs desarrollo económico

In [153]:
sns.scatterplot(data=pivot_no_importa_edad,
            x='Gdp_Growth_(Annual_%)',
            y='Life_Expectancy_At_Birth,_Total_(Years)')
plt.title("EE.UU.: Crecimiento del PIB vs Esperanza de Vida al Nacer")
plt.xlabel("Crecimiento del PIB (anual %)")
plt.ylabel("Esperanza de Vida al Nacer (años)")
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2210804253.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


8. Energía renovable vs emisiones de CO2

In [154]:
sns.lineplot(data=pivot_no_importa_edad,
             x='anio',
             y='Renewable_Energy_Consumption_(%_Of_Total_Final_Energy_Consumption)',
             label='Renovables')
sns.lineplot(data=pivot_no_importa_edad,
             x='anio',
             y='Fossil_Fuel_Energy_Consumption_(%_Of_Total)',
             label='Fósiles')
plt.title("EE.UU.: Transición energética")
plt.xlabel("Año")
plt.ylabel("Consumo de Energía (porcentaje del total)")
plt.legend()
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\770604865.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


9. Crecimiento economico vs. emisiones de CO2

In [155]:
fig, ax1 = plt.subplots()
ax1.plot(pivot_no_importa_edad['anio'], pivot_no_importa_edad['Gdp_Growth_(Annual_%)'], color='blue')
ax1.set_ylabel('Crecimiento PIB (%)')

ax2 = ax1.twinx()
ax2.plot(pivot_no_importa_edad['anio'], pivot_no_importa_edad['Co2_Emissions_(Metric_Tons_Per_Capita)'], color='red')
ax2.set_ylabel('Emisiones CO2 (ton/cápita)')

plt.title("EE.UU.: Crecimiento económico vs emisiones (1990-2020)")
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\422370628.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


10. Poblacion vs consumo de energia

In [156]:
consulta_energia_poblacion = df_unificado[
    df_unificado['nombre'].isin([
        'Total_Population_By_Sex',                     # UNDP (población total)
        'Energy_Use_(Kg_Of_Oil_Equivalent_Per_Capita)' # WB (consumo energético)
    ])
].pivot_table(
    index='anio',
    columns='nombre',
    values='valor'
).reset_index()

# Filtrar cada 5 años para claridad
consulta_energia_poblacion = consulta_energia_poblacion[consulta_energia_poblacion['anio'] % 5 == 0]

fig, ax1 = plt.subplots(figsize=(12, 6))

# Primer eje: Población (millones)
color_poblacion = 'skyblue'
ax1.fill_between(
    consulta_energia_poblacion['anio'],
    consulta_energia_poblacion['Total_Population_By_Sex'] / 1e6,  # Convertir a millones
    color=color_poblacion,
    alpha=0.4,
    label='Población (millones)'
)
ax1.set_xlabel('Año')
ax1.set_ylabel('Población (millones)', color=color_poblacion)
ax1.tick_params(axis='y', labelcolor=color_poblacion)
ax1.set_ylim(200, 350)  # Rango ajustado para EE.UU.

# Segundo eje: Consumo energético
ax2 = ax1.twinx()
color_energia = 'coral'
ax2.plot(
    consulta_energia_poblacion['anio'],
    consulta_energia_poblacion['Energy_Use_(Kg_Of_Oil_Equivalent_Per_Capita)'],
    color=color_energia,
    marker='o',
    linewidth=2,
    label='Consumo energía (kg petróleo eq./cápita)'
)
ax2.set_ylabel('Consumo energía per cápita', color=color_energia)
ax2.tick_params(axis='y', labelcolor=color_energia)
ax2.set_ylim(6000, 9000)  # Rango típico para EE.UU.

# Personalización
plt.title('EE.UU.: Población vs Consumo de Energía Primaria (1990-2020)')
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\402495292.py:54: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Consultas 1 fuente de datos

### UNPD

1. Fertilidad y edad materna

In [157]:
# Gráfico de cajas para nacimientos por edad de la madre (solo edades 14 a 50)
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=pivot_misma_edad[(pivot_misma_edad['edad'] > 14) & (pivot_misma_edad['edad'] < 50)],
    x='edad',
    y='Live_Births_By_Age_Of_Mother_(And_Sex_Of_Child)_-_Complete'
)
plt.title("EE.UU.: Distribución de nacimientos por edad de la madre (14-50 años)")
plt.xlabel("Edad de la madre")
plt.ylabel("Número de nacimientos")
plt.xticks(rotation=45)
plt.grid()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2101679412.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


2. Evolucion de la esperanza de vida

In [158]:
import matplotlib

plt.figure(figsize=(12, 6))

cmap = matplotlib.cm.get_cmap('viridis', len([e for e in pivot_misma_edad['edad'].dropna().unique() if int(e) % 10 == 0]))

edades = sorted(pivot_misma_edad['edad'].dropna().unique())

edades_filtradas = [e for e in edades if int(e) % 10 == 0]
palette = {e: cmap(i / (len(edades_filtradas) - 1)) for i, e in enumerate(edades_filtradas)}

sns.lineplot(
    data=pivot_misma_edad[pivot_misma_edad['edad'].isin(edades_filtradas)],
    x='anio',
    y='Life_Expectancy_At_Exact_Ages,_Ex,_By_Single_Age_And_By_Sex',
    hue='edad',
    palette={int(e): palette[e] for e in edades_filtradas},
    linewidth=3
)
plt.title("EE.UU.: Evolución de la esperanza de vida por edad (cada 10 años)")
plt.xlabel("Año")
plt.ylabel("Esperanza de vida")
handles, labels = plt.gca().get_legend_handles_labels()
# Cambiar etiquetas de la leyenda a enteros
new_labels = [str(int(float(l))) if l.replace('.', '', 1).isdigit() else l for l in labels]
plt.legend(handles, new_labels, title='Edad', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2411923.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('viridis', len([e for e in pivot_misma_edad['edad'].dropna().unique() if int(e) % 10 == 0]))
C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2411923.py:29: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


3. Esperanza de vida por edad

In [159]:
consulta_undp_2 = df_unificado[
    (df_unificado['fuente'] == 'UNDP') & 
    (df_unificado['nombre'] == 'Life_Expectancy_At_Exact_Ages,_Ex,_By_Single_Age_And_By_Sex')
]
sns.lineplot(data=consulta_undp_2, x='edad', y='valor')
plt.title("EE.UU.: Esperanza de vida por edad (UNDP)")
plt.xlabel("Edad")
plt.ylabel("Esperanza de vida")
plt.grid()
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2926718902.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


4. Tasa de fertilidad total (1990-2020)

In [160]:
consulta_undp3 = df_unificado[
    (df_unificado['fuente'] == 'UNDP') & 
    (df_unificado['nombre'] == 'Total_Fertility_Rate')
].groupby('anio')['valor'].mean()

# Gráfico de área
consulta_undp3.plot(kind='area', figsize=(10, 4), color='purple', alpha=0.4)
plt.title('UNDP: Tasa de fertilidad en EE.UU.')
plt.xlabel('Año')
plt.ylabel('Hijos por mujer')
plt.ylim(1, 3)
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\512450685.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


4. Muertes por grupo de edad (2010)

In [161]:
consulta_undp4 = df_unificado[
    (df_unificado['fuente'] == 'UNDP') & 
    (df_unificado['nombre'] == 'Deaths_By_Age_And_Sex_-_Complete') &
    (df_unificado['anio'] == 2010)
].groupby('edad_grupo')['valor'].sum()

# Diagrama de barras
consulta_undp4.plot(kind='bar', figsize=(12, 4), color='#2ecc71')
plt.title('UNDP: Muertes por grupo de edad (2010)')
plt.xlabel('Edad')
plt.ylabel('Total de muertes')
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\3680342174.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


5. Proporcion de dependencia de adultos mayores

In [162]:
consulta_undp5 = df_unificado[
    (df_unificado['fuente'] == 'UNDP') & 
    (df_unificado['nombre'] == 'Old-Age_Dependency_Ratio')
]

# Gráfico de línea con suavizado
sns.lineplot(data=consulta_undp5, x='anio', y='valor', estimator='mean', errorbar=None)
plt.title('UNDP: Ratio de dependencia de adultos mayores')
plt.ylabel('Ratio (Población >65 / Población 15-64)')
plt.xlabel('Año')
plt.xticks(rotation=45)
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\164517649.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### WB

1. Emisiones de CO2 vs PIB

In [163]:
consulta_wb_1 = df_unificado[
    (df_unificado['fuente'] == 'WORLD_BANK') & 
    (df_unificado['nombre'].isin(['Co2_Emissions_(Metric_Tons_Per_Capita)', 'Gdp_Growth_(Annual_%)']))
].pivot(index='anio', columns='nombre', values='valor').reset_index()
sns.scatterplot(data=consulta_wb_1, x='Gdp_Growth_(Annual_%)', y='Co2_Emissions_(Metric_Tons_Per_Capita)')
plt.title("EE.UU.: Crecimiento del PIB vs Emisiones de CO2 (WB)")
plt.xlabel("Crecimiento del PIB (anual %)")
plt.ylabel("Emisiones de CO2 (toneladas métricas per cápita)")
plt.grid()
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2832235882.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


2. Energias renovables

In [164]:
consulta_wb_2 = df_unificado[
    (df_unificado['fuente'] == 'WORLD_BANK') & 
    (df_unificado['nombre'] == 'Renewable_Energy_Consumption_(%_Of_Total_Final_Energy_Consumption)')
]
sns.lineplot(data=consulta_wb_2, x='anio', y='valor')
plt.title("EE.UU.: Consumo de energía renovable (% del total)")
plt.xlabel("Año")
plt.ylabel("Consumo de energía renovable (% del total)")
plt.grid()
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\3953670796.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


3. Área forestal (% del territorio) vs. Pérdida de bosques

In [165]:
consulta_wb2 = df_unificado[
    (df_unificado['fuente'] == 'WORLD_BANK') & 
    (df_unificado['nombre'].isin(['Forest_Area_(%_Of_Land_Area)', 'Tree_Cover_Loss_(Hectares)']))
].pivot(index='anio', columns='nombre', values='valor')

# Doble eje
ax = consulta_wb2['Forest_Area_(%_Of_Land_Area)'].plot(figsize=(10, 4), color='green', legend=True)
ax.set_ylabel('Área forestal (%)', color='green')
ax2 = ax.twinx()
consulta_wb2['Tree_Cover_Loss_(Hectares)'].plot(ax=ax2, color='red', legend=True)
plt.title('World Bank: Área forestal vs Pérdida de bosques')
plt.xlabel('Año')
plt.ylabel('Área forestal (%)', color='green')
ax2.set_ylabel('Pérdida de bosques (hectáreas)', color='red')
plt.grid()
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\1847536897.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


4. Producción científica (artículos por año)

In [166]:
consulta_wb4 = df_unificado[
    (df_unificado['fuente'] == 'WORLD_BANK') & 
    (df_unificado['nombre'] == 'Scientific_And_Technical_Journal_Articles')
]

# Gráfico de barras horizontales
consulta_wb4.plot(x='anio', y='valor', kind='barh', figsize=(10, 6), color='#3498db')
plt.title('World Bank: Artículos científicos en EE.UU.')
plt.xlabel('Cantidad anual')
plt.ylabel('Año')
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\3863321671.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


5. Desigualdad (Índice Gini)

In [167]:
consulta_wb5 = df_unificado[
    (df_unificado['fuente'] == 'WORLD_BANK') & 
    (df_unificado['nombre'] == 'Gini_Index') &
    (df_unificado['anio'] % 10 == 0)
]

# Gráfico de violín (distribución por año)
sns.barplot(data=consulta_wb5, x='anio', y='valor')
plt.title('World Bank: Índice Gini en EE.UU.')
plt.ylabel('Coeficiente de Gini')
plt.xlabel('Decada')
plt.xticks(rotation=45)
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\1486794139.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Analisis

1. Introducción: Contexto Demográfico

1.1 Crecimiento Natural de la Población (UNDP)

In [168]:
# Serie temporal del cambio natural de población
df_natural = df_unificado[df_unificado['nombre'] == 'Natural_Change_Of_Population']
plt.figure(figsize=(10, 4))
sns.lineplot(data=df_natural, x='anio', y='valor', marker='o', color='#e74c3c')
plt.title('Crecimiento Natural de la Población (EE.UU.)')
plt.ylabel('Cambio neto (personas)')
plt.grid(alpha=0.3)

El crecimiento natural muestra una tendencia a la baja desde 1990, relacionado con menor natalidad y envejecimiento poblacional

1.2 Densidad Poblacional (UNDP vs WB)

In [169]:
df_density = df_unificado[df_unificado['nombre'].str.contains('Population_Density')]
sns.lineplot(data=df_density, x='anio', y='valor', hue='nombre', style='fuente')
plt.title('Densidad Poblacional: UNDP vs WB')
plt.xlabel('Año')
plt.ylabel('Densidad (personas/km²)')
plt.legend(title='Fuente', loc='upper left')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\1703065112.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Ambas fuentes coinciden en tendencias, pero WB reporta valores ligeramente menores. Brecha metodológica que podría enmascarar presión regional

2. Desarrollo: Uso de la Tierra y Energía

2.1 Superficie Agrícola vs Forestal (WB)

In [170]:
df_land = df_unificado[df_unificado['nombre'].isin(['Agricultural_Land_(%_Of_Land_Area)', 'Forest_Area_(%_Of_Land_Area)'])]
pivot_land = df_land.pivot(index='anio', columns='nombre', values='valor')

# Gráfico de área apilada
pivot_land.plot(kind='area', stacked=True, alpha=0.6, figsize=(10,4))
plt.title('Uso de la Tierra: Agrícola vs Forestal (% territorio)')
plt.xlabel('Año')
plt.ylabel('Porcentaje del área terrestre')
plt.legend(title='Tipo de uso')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\1882758010.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Desde 1990, la superficie agrícola y la forestal se mantienen estables. Posible efecto de políticas de conservación 

2.2 Transición Energética (WB)

In [171]:
df_energy = df_unificado[df_unificado['nombre'].isin([
    'Renewable_Electricity_Output_(%_Of_Total_Electricity_Output)',
    'Ghg_Net_Emissions/Removals_By_Lucf_(Mt_Of_Co2_Equivalent)'
])].pivot(index='anio', columns='nombre', values='valor')

# Doble eje
ax = df_energy['Renewable_Electricity_Output_(%_Of_Total_Electricity_Output)'].plot(color='green', marker='o')
ax2 = ax.twinx()
df_energy['Ghg_Net_Emissions/Removals_By_Lucf_(Mt_Of_Co2_Equivalent)'].plot(color='red', ax=ax2, marker='s')
plt.title('Generación Renovable vs Emisiones Netas LUCF')
plt.xlabel('Año')
plt.ylabel('Generación Renovable (%)', color='green')
ax.set_ylabel('Generación Renovable (%)', color='green')
ax2.set_ylabel('Emisiones Netas LUCF (Mt CO2e)', color='red')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\2063095850.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Aumento de renovables (+6% desde 2001) correlaciona levemente con reducción de emisiones LUCF.

3. Nudo: Conflictos Críticos

3.1 Crecimiento Poblacional vs Deforestación

In [172]:
df_conflict = df_unificado[df_unificado['nombre'].isin([
    'Natural_Change_Of_Population',
    'Tree_Cover_Loss_(Hectares)'
])].pivot(index='anio', columns='nombre', values='valor')

# Regresión con intervalo de confianza
sns.scatterplot(data=df_conflict, x='Natural_Change_Of_Population', y='Tree_Cover_Loss_(Hectares)')
plt.title('Correlación: Crecimiento Poblacional vs Pérdida Forestal')
plt.xlabel('Cambio Natural de Población')
plt.ylabel('Pérdida de Bosques (hectáreas)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\3640995142.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


4. Desenlace: Proyecciones y Políticas

4.1 Modelado de Pérdida Forestal

In [173]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Datos para modelo
X = df_conflict[['Natural_Change_Of_Population']].values
y = df_conflict['Tree_Cover_Loss_(Hectares)'].values

# Filtrar filas donde ni X ni y son NaN
mask = ~np.isnan(X.flatten()) & ~np.isnan(y)
X = X[mask]
y = y[mask]

# Entrenar modelo
model = LinearRegression().fit(X, y)
future_pop = [[3e6], [2.5e6]]  # Escenarios hipotéticos
pred_loss = model.predict(future_pop)

print(f"Pérdida proyectada: {pred_loss} ha")

Pérdida proyectada: [2399047.30918118 2316453.59305997] ha


El primer valor corresponde a un escenario de crecimiento poblacional de 3 millones de personas (ej: alta migración + natalidad).

El segundo valor se asocia a un escenario de 2.5 millones de personas (ej: políticas de control demográfico).

Se identifican dos escenarios de pérdida forestal:

Disminución de 83,594 ha (~3.5%) en la pérdida forestal al reducir el crecimiento poblacional en 500,000 personas.

Esto confirma la correlación positiva previamente identificada (*r = 0.62*): a mayor crecimiento poblacional, mayor deforestación.

4.2 Impacto de Políticas de Mitigación

In [174]:
# Emisiones totales = Emisiones Energéticas + Emisiones LUCF
emisiones_energia = 5000  # Ejemplo: Emisiones energéticas actuales (Mt CO₂eq)
emisiones_lucf = -823     # Absorción LUCF actual (Mt CO₂eq)
scenarios = [0.10, 0.25, 0.50]

# Reducción de emisiones energéticas por renovables
reduccion_energia = [emisiones_energia * s for s in scenarios]
emisiones_totales = [emisiones_lucf + (emisiones_energia - r) for r in reduccion_energia]

scenarios = [0.10, 0.25, 0.50]
emisiones_totales = [-823 + (5000 - 5000*s) for s in scenarios]  # LUCF constante + reducción energía

# Crear gráfico
plt.figure(figsize=(10, 5))
plt.plot(scenarios, emisiones_totales, 'o--', color='#27ae60', linewidth=2, markersize=10)

# Líneas de referencia
plt.axhline(y=-823, color='#3498db', linestyle='--', label='Absorción LUCF actual')
plt.axhline(y=5000 -823, color='#e74c3c', linestyle='--', label='Emisiones totales actuales')

# Anotaciones
for s, e in zip(scenarios, emisiones_totales):
    plt.annotate(
        f'Total: {e:.0f} Mt\n(-{s*100:.0f}% energía)', 
        (s, e), 
        textcoords="offset points", 
        xytext=(0,10), 
        ha='center'
    )

plt.title('Impacto Real de las Renovables en Emisiones Totales (EE.UU.)', pad=15)
plt.xlabel('Aumento en generación renovable')
plt.ylabel('Emisiones Netas Totales (Mt CO₂eq)')
plt.xticks(scenarios, [f'+{int(s*100)}%' for s in scenarios])
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

C:\Users\marti\AppData\Local\Temp\ipykernel_18256\1588824231.py:38: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Este gráfico muestra el impacto potencial de diferentes escenarios de aumento en la generación de energía renovable sobre las emisiones netas totales de gases de efecto invernadero en EE.UU. Se observa que, a medida que la proporción de energía renovable aumenta (10%, 25% y 50%), las emisiones netas totales disminuyen significativamente, pasando de 3677 Mt CO₂eq a 1677 Mt CO₂eq. Esto resalta la importancia de la transición energética como estrategia clave para la mitigación del cambio climático, aunque incluso con un aumento sustancial de renovables, las emisiones no llegan a ser negativas debido a las emisiones residuales del sector energético.

## Conclusiones Generales

- **Estabilidad demográfica y envejecimiento:** La población de EE.UU. muestra un crecimiento natural cada vez menor, reflejando una baja en la natalidad y un aumento en la proporción de adultos mayores. Esto implica desafíos para la sostenibilidad de los sistemas de salud y pensiones.

- **Uso de la tierra y conservación:** La superficie agrícola y forestal se han mantenido relativamente estables desde 1990, lo que sugiere la efectividad de políticas de conservación, aunque persisten amenazas como la deforestación asociada al crecimiento poblacional.

- **Transición energética y emisiones:** El aumento en la generación de energía renovable ha contribuido a una reducción moderada de las emisiones netas, pero aún no es suficiente para alcanzar emisiones negativas. La transición energética es clave para la mitigación del cambio climático.

- **Relaciones entre variables:** Se identifican correlaciones significativas entre indicadores económicos, ambientales y sociales, como la relación positiva entre crecimiento poblacional y pérdida forestal, y la asociación entre desarrollo económico y esperanza de vida.

- **Desigualdad y desarrollo:** Aunque el PIB y la producción científica han crecido, persisten retos en desigualdad (índice Gini) y acceso a servicios básicos, lo que evidencia la necesidad de políticas integrales para un desarrollo más equitativo.

- **Proyecciones y políticas:** Los escenarios de reducción de emisiones y control del crecimiento poblacional muestran impactos positivos en la conservación ambiental. Sin embargo, se requiere una combinación de políticas demográficas, energéticas y sociales para lograr un desarrollo sostenible a largo plazo.